In [1]:
import requests
from html import unescape
import re
from bs4 import BeautifulSoup

import sys
sys.path.append('../helpers')
from preprocessing import preprocess_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:

url = 'https://www.bbc.com/sitemaps/https-index-com-news.xml'
res = requests.get(url)
if res.status_code != 200:
    print('ohho')

soup = BeautifulSoup(res.text, 'xml')

sitemap_tags = soup.find_all('sitemap')

sitemaps = []
for sitemap in sitemap_tags:
    loc = sitemap.find('loc')
    sitemaps.append(loc.text)

urls = []
regex_1 = r"^https://www.bbc.com/news/.*$"
regex_2 = r"^https://www.bbc.com/sport/.*$"
for sitemap in sitemaps:
    response = requests.get(sitemap)
    soup = BeautifulSoup(response.text, 'xml')
    url_tags = soup.find_all('url')
    
    for url in url_tags:
        loc = url.find('loc')
        if re.match(regex_1, loc.text) or re.match(regex_2, loc.text):
            urls.append(loc.text)

In [3]:
def url_to_text(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print('ohho')
            return None
        
        soup = BeautifulSoup(response.text, 'html.parser')
        full_article = ''
        text = []
        heading = soup.select('h1')[0].text.strip()
        text.append(heading)
        
        for p in soup.select('p'):
            text.append(unescape(p.text))
        full_article = ' '.join(text)
        
        full_article_part = full_article[:600]
        article_clean = preprocess_text(full_article)
        return (url,heading, article_clean,full_article_part )
    
    except requests.exceptions.Timeout as ee :
        print('TO')
        return None


In [4]:
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="SearchEngine",
    user="postgres",
    password="2580"
)



# Open a cursor to perform database operations
cur = conn.cursor()


for url in urls:
    try:
        url,header, text ,content_raw = url_to_text(url)
        data = (url,header, text, content_raw)
        sql = "INSERT INTO sites (url,header,content,content_raw) VALUES (%s,%s,%s,%s)"
        cur.execute(sql, data)
        conn.commit()
    except TypeError as ee :
        continue


TO
TO
ohho
TO
TO
TO
TO
ohho
ohho
TO
TO
ohho
TO
TO
TO
TO
TO
TO
TO
TO
ohho
TO
ohho
TO
TO
TO
TO
TO
TO
TO
TO
ohho
TO


In [5]:
cur.close()
conn.close()